In [1]:
library(mediation)
library(multilevel)
library(lavaan)

Loading required package: MASS

Loading required package: Matrix

Loading required package: mvtnorm

Loading required package: sandwich

mediation: Causal Mediation Analysis
Version: 4.5.0


Loading required package: nlme

Warning message:
“package ‘lavaan’ was built under R version 4.2.3”
This is lavaan 0.6-17
lavaan is FREE software! Please report any bugs.



In [2]:
# Source: https://library.virginia.edu/data/articles/introduction-to-mediation-analysis
# X: grades; Y: happiness; M: self-esteem
myData = read.csv('http://static.lib.virginia.edu/statlab/materials/data/mediationData.csv')
head(myData)

,X,M,Y
,<int>,<int>,<int>
1,6,5,6
2,7,5,5
3,7,7,4
4,8,4,8
5,4,3,5
6,4,4,7


## Baron & Kenny’s 4-step method

In [3]:
# Step 1: Total effect = Path c = 0.3961***
model.0 = lm(Y ~ X, data=myData)
summary(model.0)


Call:
lm(formula = Y ~ X, data = myData)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.0262 -1.2340 -0.3282  1.5583  5.1622 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   2.8572     0.6932   4.122 7.88e-05 ***
X             0.3961     0.1112   3.564 0.000567 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.929 on 98 degrees of freedom
Multiple R-squared:  0.1147,	Adjusted R-squared:  0.1057 
F-statistic:  12.7 on 1 and 98 DF,  p-value: 0.0005671


In [4]:
# Step 2: Path a = 0.56102***
model.XM = lm(M ~ X, data=myData)
summary(model.XM)


Call:
lm(formula = M ~ X, data = myData)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.3046 -0.8656  0.1344  1.1344  4.6954 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.49952    0.58920   2.545   0.0125 *  
X            0.56102    0.09448   5.938 4.39e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.639 on 98 degrees of freedom
Multiple R-squared:  0.2646,	Adjusted R-squared:  0.2571 
F-statistic: 35.26 on 1 and 98 DF,  p-value: 4.391e-08


In [5]:
# Step 3 & 4: Path b + Path c'
#             Path b = 0.6355***
#             Path c'= 0.0396
model.MY.XY = lm(Y ~ X + M, data=myData)
summary(model.MY.XY)


Call:
lm(formula = Y ~ X + M, data = myData)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.7631 -1.2393  0.0308  1.0832  4.0055 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   1.9043     0.6055   3.145   0.0022 ** 
X             0.0396     0.1096   0.361   0.7187    
M             0.6355     0.1005   6.321 7.92e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.631 on 97 degrees of freedom
Multiple R-squared:  0.373,	Adjusted R-squared:  0.3601 
F-statistic: 28.85 on 2 and 97 DF,  p-value: 1.471e-10


Interpretation:
- The total effect model shows a significant positive relationship between grades(X) and happiness(Y).
    - Path c = 0.3961***
- The path a model shows grades(X) positively correlate with self-esteem(M) and the relationship is statistically significant.
    - Path a = 0.56102***
- Self-esteem(M) positively predicts happiness(Y) when controlling grades(X). At the same time, the relationship between grades(X) and happiness(Y) is no longer significant.
    - Path b = 0.6355***
    - Path c'= 0.0396
- This suggests that self-esteem(M) completely mediates the relationship between grades(X) and happiness(Y).
    - However, this method cannot directly evaluate the significance of the indirect effect (i.e. ab≠0) - we need a formal test for a statistical verification.

## Sobel Test

Determine the significance of the indirect effect.

In [6]:
# 手动计算: https://www.spss-tutorials.com/sobel-test-what-is-it/
alpha = summary(model.XM)$coefficients["X","Estimate"]
se_alpha = summary(model.XM)$coefficients["X","Std. Error"]
beta = summary(model.MY.XY)$coefficients["M","Estimate"]
se_beta = summary(model.MY.XY)$coefficients["M","Std. Error"]

z.statistic = (alpha * beta)/(sqrt(alpha*alpha*se_beta*se_beta + beta*beta*se_alpha*se_alpha))
p.value = 2*(1-pnorm(z.statistic))  # two-tailed

sprintf("indirect effect: %s", alpha*beta)
sprintf("z-statistic: %s", z.statistic)
sprintf("p-value: %s", p.value)

[1] "indirect effect: 0.356522196728078"

[1] "z-statistic: 4.32789111803173"

[1] "p-value: 1.5054385481994e-05"

In [7]:
# 使用函数
# library(multilevel)
sobel(myData$X, myData$M, myData$Y)

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),2.8572046,0.6932130,4.121684,7.875771e-05
pred,0.3961261,0.1111598,3.563574,5.671128e-04
,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),1.90426882,0.6054585,3.145168,2.203831e-03
pred,0.03960392,0.1096484,0.361190,7.187429e-01
med,0.63549459,0.1005339,6.321194,7.922642e-09
,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),1.4995183,0.58919773,2.545017,1.248749e-02
pred,0.5610153,0.09448048,5.937897,4.391362e-08


## 使用mediation

In [8]:
res.1 = mediate(model.XM, model.MY.XY, treat='X', mediator='M', 
                boot=TRUE, sims=1000)
summary(res.1)

Running nonparametric bootstrap





Causal Mediation Analysis 

Nonparametric Bootstrap Confidence Intervals with the Percentile Method

               Estimate 95% CI Lower 95% CI Upper p-value    
ACME             0.3565       0.2142         0.53  <2e-16 ***
ADE              0.0396      -0.1984         0.29   0.752    
Total Effect     0.3961       0.1602         0.63   0.004 ** 
Prop. Mediated   0.9000       0.4927         2.11   0.004 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Sample Size Used: 100 


Simulations: 1000 


## 使用lavaan

In [9]:
# Step 1: Specify mediation model
# The lavaan syntax
model = "
# Path c' (direct effect)
Y ~ c*X
# Path a
M ~ a*X
# Path b
Y ~ b*M
# Indirect effect (a*b)
ab := a*b
"

In [10]:
# Step 2: Estimate model - delta method (by default)
fitmod = sem(model, data=myData)
# Step 3: Request summary
summary(fitmod, fit.measures=TRUE, rsquare=TRUE)

lhs,op,rhs,label,exo,est,se,z,pvalue
<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
Y,~,X,c,0,0.03960392,0.10799119,0.3667329,7.138183e-01
M,~,X,a,0,0.56101532,0.09353090,5.9981813,1.995397e-09
Y,~,M,b,0,0.63549459,0.09901445,6.4182005,1.378946e-10
Y,~~,Y,,0,2.58142482,0.36506860,7.0710678,1.537437e-12
M,~~,M,,0,2.63306954,0.37237227,7.0710678,1.537437e-12
X,~~,X,,1,3.00990000,0.00000000,NA,NA
ab,:=,a*b,ab,0,0.35652220,0.08135460,4.3823238,1.174201e-05
Y,r2,Y,,0,0.37299924,NA,NA,NA
M,r2,M,,0,0.26458788,NA,NA,NA


In [11]:
# Step 2: Estimate model - percentile bootstrapping method
fitmod2 = sem(model, data=myData, se="bootstrap", bootstrap=1000)
# Ste[pRequest percentile bootstrap 95% confidence intervals
parameterEstimates(fitmod2, ci=TRUE, level=0.95, boot.ci.type="perc")

lhs,op,rhs,label,est,se,z,pvalue,ci.lower,ci.upper
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Y,~,X,c,0.03960392,0.12755884,0.3104757,7.561992e-01,-0.2000781,0.2901507
M,~,X,a,0.56101532,0.09658860,5.8082973,6.311139e-09,0.3874452,0.7691867
Y,~,M,b,0.63549459,0.10507922,6.0477665,1.468676e-09,0.4311823,0.8426629
Y,~~,Y,,2.58142482,0.32884071,7.8500767,4.218847e-15,1.9048312,3.1598721
M,~~,M,,2.63306954,0.34658373,7.5972104,3.019807e-14,1.9150087,3.3124284
X,~~,X,,3.00990000,0.00000000,NA,NA,3.0099000,3.0099000
ab,:=,a*b,ab,0.35652220,0.08216977,4.3388485,1.432312e-05,0.2168658,0.5321631
